In [2]:
import qsharp
import matplotlib.pyplot as plt
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler=StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

scaler.fit(Y_train)

Y_train_scaled = scaler.transform(Y_train)
Y_test_scaled = scaler.transform(Y_test)

In [3]:
%%qsharp

open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;

operation ZZFeatureMap(qs : Qubit[], data : Double[]) : Unit {
    let numberOfQubits = Length(qs);
    if (numberOfQubits != Length(data)) {
        fail "Data length must be equal to the number of qubits.";
    }
    for i in 0..numberOfQubits-1 {
        // Apply Z rotation to each qubit
        Rz(2.0 * PI() * data[i], qs[i]);
    }
    for i in 0..numberOfQubits-1 {
        for j in i+1..numberOfQubits-1 {
            // Apply ZZ interaction between each pair of qubits
            let theta = 2.0 * PI() * data[i] * data[j];
            CNOT(qs[i], qs[j]);
            Rz(theta, qs[j]);
            CNOT(qs[i], qs[j]);
        }
    }
}


operation RealAmplitudes(qubits : Qubit[], parameters : Double[]) : Unit {
    // Apply layer of rotation gates
    for idx in IndexRange(qubits) {
        Ry(parameters[idx], qubits[idx]);
    }

    // Apply a layer of CNOT gates
    for idx in 0 .. Length(qubits)-2 {
        for i in idx+1 .. Length(qubits)-1{
            CNOT(qubits[idx], qubits[i]);
        }
    }


    // Apply another layer of rotation gates
    for idx in IndexRange(qubits) {
        Ry(parameters[idx + Length(qubits)], qubits[idx]);
    }
}

operation quantumVariationalClassifier (qubits: Qubit[], x : Double[], θ : Double[]) : Unit {
    ZZFeatureMap(qubits, x);
    RealAmplitudes(qubits, θ);
}



In [ ]:
def cost(weights, data, labels):
    # Make predictions using the quantum neural network, labels are the expected/real classification for the data
    predictions = quantumVariationalClassifier(data, x, weights)
    mse = qml.mean_squared_error(labels, predictions)
    return mse

optimizer = qml.AdamOptimizer(stepsize=0.01)

steps = 100

#probably need to make this into a list or array to match theta type in other functions
weights = np.random.normal(0, 1, (4, 2))

# Train the model
for i in range(steps):
    gradients = qml.grad(cost, argnum=0)(weights, X_train_scaled, y_train)

    # Update the weights
    optimizer.step(gradients, weights)

    # Print the cost
    if (i + 1) % 10 == 0:
        print(f'Step {i + 1}: cost = {cost(weights, X_train_scaled, y_train):.4f}')

predictions = quantumVariationalClassifier(weights, X_test_scaled)

accuracy = qml.accuracy(predictions, y_test)

print(f'Test accuracy: {accuracy:.2f}')

plt.scatter(y_test, predictions)

# Add a diagonal line
x = np.linspace(0, 3, 4)
plt.plot(x, x, '--r')

# Add axis labels and a title
plt.xlabel('True labels')
plt.ylabel('Predictions')
plt.title('Quantum Neural Network')

# Show the plot
plt.show()



